In [1]:
import sys
sys.path.append("InfectionModel/contagion_sim/")

In [2]:
import numpy as np
import pandas as pd
import tqdm

from continuous_fast import MultisimRankModel

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
contacts = pd.read_csv("contacts_t_12.csv.gz",index_col=0)

/home/fmazza/app/miniconda3/envs/myenv/lib/python3.8/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
observations = pd.read_csv("observ_t_12.csv.gz",index_col=0)

### Description

We have 12 days of contacts and a variable number of observations up until day 11

We want to obtain the risk prediction for the nodes on day 12

In [6]:
observations

,i,state,t
0,3032,1,7
1,6865,0,7
2,2868,0,7
3,518,0,7
4,7375,0,7
...,...,...,...
47,2207,0,11
48,5215,0,11
49,6637,0,11
50,5255,0,11


In [7]:
contacts

,i,j,t
0,0,3,0
1,0,2,0
2,0,1,0
3,0,1540,0
4,0,7583,0
...,...,...,...
1320789,9999,2726,12
1320790,9999,7050,12
1320791,9999,6990,12
1320792,9999,9242,12


In [8]:
N = max(contacts["i"])+1
n_days = max(contacts["t"])
p_infection = 0.02
mu = 1/50
P_INIT_INF = 1/N

N_SIMS = 1000

In [9]:
model = MultisimRankModel(N, p_infection, mu, n_days, N_SIMS, contacts_directed=True, tqdm=tqdm.tqdm)

### Add contacts
Replacing the index in the contacts

In [10]:
replace_name_map = {"i":"a", "j":"b"}

In [11]:
for day, con_day  in contacts.groupby("t"):
    model.add_contacts_day(day, con_day.rename(mapper=replace_name_map, axis=1).reset_index(drop=True))

### Add observations

Add the observations for each day, formatting them for ViraTrace

In [12]:
for day, observ_day in observations.groupby("t"):
    #print(observ_day)
    positive_test_idx = observ_day[observ_day.state == 1]["i"]
    print(positive_test_idx.values)
    
    negative_test_idx = observ_day[observ_day.state == 0]["i"]
    
    tested_positive = np.full(N,False)
    if len(positive_test_idx) > 0:
        tested_positive[positive_test_idx.values] = True
    tested_negative = np.full(N,False)
    if len(negative_test_idx) > 0:
        tested_negative[negative_test_idx.values] = True

    model.set_daily_observations(day, tested_negative, tested_positive)
    print(day)


[3032]
7
[4698 6688 6118]
8
[8345 1839]
9
[5647 1085]
10
[9438 8182]
Propagation
11


### Launch simulations
Since we have no observations of the initial condition, we have to use a prior probability for each node to be infected at time 0

In [13]:
model.start_sim(P_INIT_INF)

Starting simulation...:   0%|          | 0/12 [00:00<?, ?it/s]

Setting random initial condition


Day:1	S:99%	I:0%	R:0%:   8%|▊         | 1/12 [00:03<00:43,  3.92s/it]

Day:2	S:99%	I:0%	R:0%:  17%|█▋        | 2/12 [00:07<00:38,  3.88s/it]

Day:3	S:99%	I:0%	R:0%:  25%|██▌       | 3/12 [00:11<00:35,  3.91s/it]

...

Day:8	S:99%	I:0%	R:0%:  67%|██████▋   | 8/12 [00:31<00:15,  3.94s/it]

Day:9	S:99%	I:0%	R:0%:  75%|███████▌  | 9/12 [00:35<00:11,  3.95s/it]

Day:10	S:99%	I:0%	R:0%:  83%|████████▎ | 10/12 [00:39<00:07,  3.94s/it]

Day:11	S:99%	I:0%	R:0%:  92%|█████████▏| 11/12 [00:43<00:03,  3.92s/it]

Day:12	S:99%	I:0%	R:0%: 100%|██████████| 12/12 [00:46<00:00,  3.89s/it]


### Calculate risk

In [17]:
risk = pd.Series(model.I.sum(1))

risk = (risk / risk.max()).sort_values()

In [18]:
sorted_risk = risk.sort_values(ascending=False)

In [19]:
sorted_risk.to_csv("nodes_risk.csv")